### Load source code

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import sys
sys.path.append('../')

In [19]:
import source.players as pl
import source.parsers as parsers
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.runner as runner
import source.util as util
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.players.holmes as holmes
import source.players.sad as sad
import source.players.b2bw2w as b2bw2w
import source.players.belief_max as bm
import source.players.dmd as dmd
import source.players.bayesian_approximator

import concurrent.futures
import pkgutil
import logging
import sys
from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time
import os
import shutil
import re
from copy import deepcopy, copy
import datetime
from contextlib import redirect_stdout
from random import shuffle

In [6]:
# run once

lvl = logging.DEBUG
logger = logging.getLogger("source")
logger.setLevel(lvl)
#ch = logging.StreamHandler(sys.stdout)
ch = logging.FileHandler("status", mode='w')
ch.setLevel(lvl)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [7]:
lvl = logging.WARNING
logger.setLevel(lvl)
ch.setLevel(lvl)

In [201]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(environment)
reload(runner)
reload(parsers)
reload(game)
reload(holmes)
reload(sad)
reload(b2bw2w)
reload(util)
reload(bm)
reload(dmd)

<module 'source.players.dmd' from '..\\source\\players\\dmd.py'>

### Set batch path, batch folder need not to already exist

In [202]:
experiments_folder = "../NewExperiments"
exp_subfolder = "/Full feedback vs MAB" #insert folder where batch is
batch_name = "fb_vs_sto_4(problematic)" #insert batch name
batch_file = batch_name + ".csv"
batch_path = experiments_folder + exp_subfolder + "/" + batch_file

### Parsing the csv to generate a batch containing the configurations and running them

In [203]:
b = runner.Batch(batch_path, experiments_folder + exp_subfolder)

In [204]:
if os.path.isdir(experiments_folder + exp_subfolder + "/" + batch_name):
    shutil.rmtree(experiments_folder + exp_subfolder + "/" + batch_name)
b.parse_batch()

In [205]:
batches = []
batches.append(b)

In [206]:
with concurrent.futures.ProcessPoolExecutor(None) as executor:
    futures = {}
    logger.info(str(datetime.datetime.now()))
    for b in batches:
        b.run(futures, executor, n=100)

### Collecting results of the runs

In [207]:
with open('progress', 'w') as f:
    with redirect_stdout(f):
        for b in batches:
            b.collect(futures)
            confs = b.configurations
            print(b.results_folder_path)
            dlist = [{"name": c.game.players[0].__class__.name,
                      "avgs": c.stats["exp_regret"], 
                      "lb": c.stats["lb_exp_regret"],
                      "ub": c.stats["ub_exp_regret"]}
                     for c in confs]
            util.plot_dicts(dlist, ylabel="$exp\_reg(t)$", xlabel="$t$", save=True, show=False, semilog=False,
                            path=b.results_folder_path,
                            title=confs[0].game.players[0].__class__.name + " vs " + confs[0].game.players[1].__class__.name)

### Checking everything was fine by running the configuration "step-by-step"

In [192]:
gm = deepcopy(batches[0].configurations[0].game)

In [193]:
e = runner.Experiment(gm)

In [194]:
for i in range(50):
    #e.run_interaction_with_observations()
    e.run_interaction()
    print(e.game.strategy_history[-1])
    #print(util.game_str(e.game, lenght=7))
    print(e.game.history[-1])
    #print(list(e.agent.belief.pr.items())[2]) 0.809,0.842,0.933,0.875,0.986,0.876 0.18-0.059-0.13-0.046-0.34-0.23

LAST PAYOFF:  0.9329999999999999
{0: (0, 0, 0, 1, 0, 0), 1: [0.1898101898101898, 0.05894105894105893, 0.13486513486513485, 0.045954045954045945, 0.33966033966033965, 0.23076923076923075]}
{1: [2], 0: [3]}
sto1-0.15-0.116-0.139-0.433-0.014-0.148 with diff loss 0.440839
sto1-0.33-0.285-0.27-0.039-0.034-0.042 with diff loss 0.10383399999999987
sto1-0.452-0.159-0.005-0.003-0.251-0.13 with diff loss 0.0674229999999999
sto1-0.02-0.171-0.292-0.187-0.151-0.179 with diff loss 0.194712
sto1-0.237-0.001-0.174-0.054-0.249-0.285 with diff loss 0.08290900000000001
sto1-0.4744744744744745-0.0890890890890891-0.11011011011011013-0.1811811811811812-0.13913913913913917-0.006006006006006007 with diff loss 0.22895195195195173
sto1-0.2672672672672673-0.2122122122122122-0.07907907907907907-0.12212212212212212-0.06606606606606608-0.25325325325325326 with diff loss 0.17732632632632628
sto1-0.177-0.316-0.198-0.099-0.104-0.106 with diff loss 0.14360099999999998
sto1-0.046-0.398-0.024-0.334-0.056-0.142 with diff 

sto1-0.02-0.171-0.292-0.187-0.151-0.179 with diff loss 0.754903
sto1-0.237-0.001-0.174-0.054-0.249-0.285 with diff loss 0.5494440000000003
sto1-0.4744744744744745-0.0890890890890891-0.11011011011011013-0.1811811811811812-0.13913913913913917-0.006006006006006007 with diff loss 1.041202202202202
sto1-0.2672672672672673-0.2122122122122122-0.07907907907907907-0.12212212212212212-0.06606606606606608-0.25325325325325326 with diff loss 0.4842982982982983
sto1-0.177-0.316-0.198-0.099-0.104-0.106 with diff loss 0.7594570000000002
sto1-0.046-0.398-0.024-0.334-0.056-0.142 with diff loss 0.8594100000000003
sto1-0.020020020020020023-0.19619619619619622-0.4914914914914915-0.18718718718718722-0.06506506506506507-0.04004004004004005 with diff loss 0.9734304304304304
sto1-0.188-0.114-0.189-0.246-0.235-0.028 with diff loss 1.1539039999999998
sto1-0.07-0.06-0.152-0.497-0.111-0.11 with diff loss 1.107601
sto1-0.006993006993006994-0.1228771228771229-0.1098901098901099-0.6083916083916084-0.02597402597402597

sto1-0.1018981018981019-0.15084915084915085-0.17882117882117884-0.25974025974025977-0.18481518481518483-0.12387612387612389 with diff loss 1.4094815184815186
sto1-0.1691691691691692-0.20520520520520522-0.036036036036036036-0.38138138138138145-0.11011011011011013-0.09809809809809811 with diff loss 1.0054934934934943
sto1-0.6963036963036964-0.15984015984015987-0.02297702297702298-0.07392607392607393-0.03796203796203797-0.008991008991008992 with diff loss 1.7224525474525483
sto1-0.231-0.125-0.164-0.055-0.066-0.359 with diff loss 2.291776
CHOSEN:  sto1-0.15-0.116-0.139-0.433-0.014-0.148
LOSSES:  {<StochasticAttacker id:1 resources:1>: [0.46649999999999997, 2], <StochasticAttacker id:1 resources:1>: [0, 0], <StochasticAttacker id:1 resources:1>: [0, 0], <StochasticAttacker id:1 resources:1>: [0.8420000000000001, 1], <StochasticAttacker id:1 resources:1>: [0, 0], <StochasticAttacker id:1 resources:1>: [0, 0], <StochasticAttacker id:1 resources:1>: [0.7262500000000001, 4], <StochasticAttacker

sto1-0.020020020020020023-0.19619619619619622-0.4914914914914915-0.18718718718718722-0.06506506506506507-0.04004004004004005 with diff loss 4.923265265265266
sto1-0.188-0.114-0.189-0.246-0.235-0.028 with diff loss 2.943311
sto1-0.07-0.06-0.152-0.497-0.111-0.11 with diff loss 2.443361000000002
sto1-0.006993006993006994-0.1228771228771229-0.1098901098901099-0.6083916083916084-0.025974025974025976-0.1258741258741259 with diff loss 2.823018981018983
sto1-0.27672327672327673-0.10789210789210787-0.35964035964035956-0.2017982017982018-0.043956043956043946-0.009990009990009988 with diff loss 3.887459540459539
sto1-0.1898101898101898-0.05894105894105893-0.13486513486513485-0.045954045954045945-0.33966033966033965-0.23076923076923075 with diff loss 4.294844155844154
sto1-0.377-0.118-0.054-0.153-0.297-0.001 with diff loss 4.208274
sto1-0.1018981018981019-0.15084915084915085-0.17882117882117884-0.25974025974025977-0.18481518481518483-0.12387612387612389 with diff loss 1.958920079920079
sto1-0.1691

sto1-0.452-0.159-0.005-0.003-0.251-0.13 with diff loss 8.062244999999999
sto1-0.02-0.171-0.292-0.187-0.151-0.179 with diff loss 4.615312999999999
sto1-0.237-0.001-0.174-0.054-0.249-0.285 with diff loss 7.596495999999997
sto1-0.4744744744744745-0.0890890890890891-0.11011011011011013-0.1811811811811812-0.13913913913913917-0.006006006006006007 with diff loss 3.7742592592592628
sto1-0.2672672672672673-0.2122122122122122-0.07907907907907907-0.12212212212212212-0.06606606606606608-0.25325325325325326 with diff loss 5.295739739739738
sto1-0.177-0.316-0.198-0.099-0.104-0.106 with diff loss 5.788882999999999
sto1-0.046-0.398-0.024-0.334-0.056-0.142 with diff loss 2.919634000000003
sto1-0.020020020020020023-0.19619619619619622-0.4914914914914915-0.18718718718718722-0.06506506506506507-0.04004004004004005 with diff loss 6.391028028028031
sto1-0.188-0.114-0.189-0.246-0.235-0.028 with diff loss 3.5331360000000003
sto1-0.07-0.06-0.152-0.497-0.111-0.11 with diff loss 4.4851490000000025
sto1-0.0069930

In [34]:
pl=gm.players[0]

In [35]:
pl.game.profiles

[<SUQR id:1 resources:1>,
 <SUQR id:1 resources:1>,
 <SUQR id:1 resources:1>,
 <FrequentistUnknownStochasticAttacker id:1 resources:1>,
 <StackelbergAttacker id:1 resources:1>,
 <SUQR id:1 resources:1>,
 <SUQR id:1 resources:1>,
 <SUQR id:1 resources:1>,
 <SUQR id:1 resources:1>,
 <SUQR id:1 resources:1>]